In [65]:
%load_ext autoreload
%autoreload 2
import os
from os.path import join as oj

import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import sys
import pickle as pkl
import face_recognition
sys.path.append('..')

import data
import style
import config
from config import *
import util

df = data.load_all_labs()
df = df.set_index('fname_id')

# get fnames
fname_nps = [f for f in sorted(os.listdir(DIR_GEN)) if 'npy' in f] # these start at 00001
fname_ids = np.array([f[:-4] for f in fname_nps])
idxs_calculated = np.array([int(x) - 1 for x in fname_ids]) # this starts at 0

# trim df to only have the relevant ids
df = df.loc[fname_ids]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
loading cached labels


# select celebrities

In [87]:
N_IMS_PER_SUBGROUP = 10
# only consider ids with more than 2 ims
d = df[df['count_with_this_id'] > 2]

# sort by how many ids each image has
d = d.sort_values('count_with_this_id', ascending=False)

# get ids for subgroups
subgroups = {}
for gender in [0, 1]:
    for race in ['Black', 'White']:
        dd = d[d['gender'] == gender]
        dd = dd[dd['race_pred'] == race]
        print(gender, race, '|', dd.id.unique().size, 'ids\t', dd.shape[0], 'photos')        
        subgroups[(gender, race)] = dd.id.unique()[:N_IMS_PER_SUBGROUP]


0 Black | 91 ids	 468 photos
0 White | 1468 ids	 8534 photos
1 Black | 157 ids	 570 photos
1 White | 841 ids	 3600 photos


In [92]:
def pair_plot(im0, im1):
    R, C = 1, 2
    plt.subplot(R, C, 1)
    plt.title('Real photo')
    util.imshow(im0)
    plt.subplot(R, C, 2)
    plt.title('Test photo')
    util.imshow(im1)
    plt.tight_layout()
    
def slight_crop(im):
    x = np.random.

In [ ]:
EXPERIMENT_DIR = oj(DIR_PROCESSED, 'projections_annotation', 'ims')
os.makedirs(EXPERIMENT_DIR, exist_ok=True)
for gender, race in tqdm(subgroups.keys()):
    # print(gender, race)
    ids = subgroups[(gender, race)]
    for i in ids[:2]:
        ims = d[d.id == i].iloc[:2]
        im0 = mpimg.imread(oj(DIR_IMS, ims.fname_final.values[0]))
        im1 = mpimg.imread(oj(DIR_IMS, ims.fname_final.values[1]))
        im1_rec = mpimg.imread(oj(DIR_GEN, ims.fname_final.values[1][:-4] + '.png'))
        pair_plot(im0, im1)
        plt.savefig(oj(EXPERIMENT_DIR, f'{gender}_{race}_{i}_real.png'), dpi=300)
        pair_plot(im0, im1_rec)
        plt.savefig(oj(EXPERIMENT_DIR, f'{gender}_{race}_{i}_fake.png'), dpi=300)
        pair_plot(im0, im0)
        plt.savefig(oj(EXPERIMENT_DIR, f'{gender}_{race}_{i}_fake.png'), dpi=300)        